# Advanced Constraints

The purpose of this tutorial is to demonstrate the advanced capabilities of the **Constraint Engine**. For an introduction to the constraints, refer to [Dataset & Constraints](https://github.com/ydataai/academy/blob/master/3%20-%20dataset-metadata/Metadata%20%26%20Constraints.ipynb).

The plan of this notebook is as follows:

1. **Rows versus Columns constraints**: we introduce the difference between rows and columns constraints and demonstrate how they can be used together.
2. **Custom Constraints**: we show how to create your own constraints based on any python function. In addition, we show an advanced example to validate constraints with time dependencies between the rows.


In [6]:
from ydata.constraints.engine import ConstraintEngine

## Toy dataset: a transactional dataset

To illustrate this tutorial, we will generate a toy dataset representing the transaction on a bank account.   
Each row represents a transaction. The dataset has only 3 columns:

1. `amount`: the amount of the transaction
2. `balance`: the current balance after the transaction is applied
3. `constant`: a constant value (for the purpose of demonstrating some constraints)

It is a very generous example, so the amount can only be positive such that your balance keeps growing! The amount is generated randomly according to normal distribution.

In [3]:
import numpy as np
import pandas as pd

from ydata.dataset import Dataset
from ydata.metadata import Metadata

def calculate_balance(df):
    df['balance'] = df['amount'].cumsum()
    return df

n = 10 ** 3
mu, sigma = 0, 0.1
data = 100 * np.abs(np.random.normal(mu, sigma, n))
df = pd.DataFrame()
df['amount'] = data
df['constant'] = 10
df = calculate_balance(df)
df = df.reset_index()

In [60]:
df

,index,amount,constant,balance
0,0,15.134607,10,15.134607
1,1,4.135724,10,19.270331
2,2,20.515678,10,39.786009
3,3,3.057633,10,42.843642
4,4,4.660475,10,47.504117
...,...,...,...,...
995,995,7.668661,10,7427.040967
996,996,7.033443,10,7434.074410
997,997,2.299090,10,7436.373500
998,998,24.251009,10,7460.624508


# Rows versus Columns constraints

Generally, when it comes to validate constraints on a tabular dataset, there are two different aspects:
1. the constraints that apply to the data points (e.g.: each row should have the column `balance` positive)
2. the constraints that apply to quantity linked to the columns (e.g.: the average of the column `balance` must be positive)

They are conceptually different because the **rows** constraints are applied to each data points independently while the **columns** constraint are applied to an aggregation of all rows and represent a **domain** validation.

Both are useful to validate the quality of your data and perfectly works together seamlessly. However, there is conceptual difference on the output: the **rows** constraints return a mask indicating for each rows if the constraint is violated; the **columns** constraints returns a boolean indicating if a column violates the constraint or not.

In the example below, add two columns constraints during the metadata calculation:
- the mean of the columns `amount` and `constant` must be between 7 and 10
- the standard deviation of the columns `amount` and `constant` must be between 0 and 8

In [28]:
c1 = MeanBetween(lower_bound=7, upper_bound=10, columns=['amount', 'constant'])
c2 = StandardDeviationBetween(lower_bound=0, upper_bound=8, columns=['amount', 'constant'])

m = Metadata(original, constraints=[c1, c2])
m = m(original)

[########################################] | 100% Completed | 101.72 ms
[########################################] | 100% Completed | 102.63 ms
[########################################] | 100% Completed | 102.34 ms
[########################################] | 100% Completed | 204.14 ms
sample size: 10538
[########################################] | 100% Completed | 101.38 ms
sample size: 10538
[########################################] | 100% Completed | 102.23 ms
[########################################] | 100% Completed | 101.23 ms
[########################################] | 100% Completed | 101.58 ms
[########################################] | 100% Completed | 100.99 ms
[########################################] | 100% Completed | 205.82 ms


It is possible to have a summary of the constraints directly from the `Metadata` object.    

In the field `violation_per_constraint`, it is possible to access more information for each constraint. For instance, `StandardDeviationBetween` shows that 1 column out of 2 columns have been violated.

In [29]:
m.summary['constraints']

{'rows_violation_count': 0,
 'rows_violation_ratio': 0.0,
 'violation_per_constraint': {'MeanBetweenRange on columns [amount, constant]': {'column_violation_count': 0,
   'column_violation_ratio': 0.0,
   'violated_columns': [],
   'validation_time': 0.12},
  'StandardDeviationBetween on columns [amount, constant]': {'column_violation_count': 1,
   'column_violation_ratio': 0.5,
   'violated_columns': ['constant'],
   'validation_time': 0.12}}}

Finally, it is possible to have a pretty print summary of the constraint engine validation:

In [26]:
m._constraint_engine


Constraint Engine 
 
Number of Constraints: 2
Number of Constraints Violated: 1 (50.0%)

MeanBetweenRange on columns [amount, constant]: 0 (0.0%)
StandardDeviationBetween on columns [amount, constant]:  1 violated (50.0%) 

## Columns constraints

In this section, we provide multiple examples of columns constraints.

In [12]:
from ydata.constraints.columns import MeanBetween, StandardDeviationBetween, QuantileBetween, UniqueValuesBetween, NullValuesCountLowerThan, SumLowerThan, Constant

We would like to make sure that the mean value of the columns `amount` and the column `constant` is between 7 and 10 (included):

In [13]:
c = MeanBetween(lower_bound=7, upper_bound=10, columns=['amount', 'constant'])
c.validate(dataset)

sample size: 10538


,amount,constant
0,True,True


We would like to make sure that the standard deviation is comprised between 0 (excluded) and 8:

In [16]:
c = StandardDeviationBetween(lower_bound=0, upper_bound=8, columns=['amount', 'constant'])
c.validate(dataset)

sample size: 10538


,amount,constant
0,True,False


Because the column `constant` is constant, its standard deviation is exactly 0 and thus, the column violate the constraint!

It is also possible to constraint any quantile to any particular interval. For instance, the following constraint check that the 25% centile is between 3 and 3.2:

In [18]:
c = QuantileBetween(quantile=0.25, lower_bound=3, upper_bound=3.2, columns='amount')
c.validate(dataset)

sample size: 10538


,amount
0,False


It is possible check that the number of unique value in a column belong to a given interval. The following example is equivalent to check that a column is constant as it requires the number of unique values to be between 0 (excluded) and 1:

In [19]:
c = UniqueValuesBetween(lower_bound=0, upper_bound=1, columns='constant')
c.validate(dataset)

sample size: 10538


,constant
0,True


For such a trivial example, we directly provide a constraint `Constant`:

In [21]:
c = Constant(columns='constant')
c.validate(dataset)

sample size: 10538


,constant
0,True


Finally, it is possible check that the sum of a column is lower than a value value:

In [22]:
c = SumLowerThan(value=10001, columns='constant')
c.validate(dataset)

sample size: 10538


,constant
0,True


## Mixing Rows and Columns constraints

The Constraint Engine accepts both type of constraints:

In [32]:
from ydata.constraints.rows import GreaterThan

# Rows constraint
c1 = GreaterThan(columns=['amount', 'balance'], value=1)

# Column constraint
c2 = MeanBetween(lower_bound=7, upper_bound=10, columns=['amount', 'constant'])
c3 = StandardDeviationBetween(lower_bound=0, upper_bound=8, columns=['amount', 'constant'])

m = Metadata(original, constraints=[c1, c2, c3])
m = m(original)

[########################################] | 100% Completed | 101.31 ms
[########################################] | 100% Completed | 101.80 ms
[########################################] | 100% Completed | 104.47 ms
[########################################] | 100% Completed | 204.66 ms
sample size: 10538
[########################################] | 100% Completed | 101.30 ms
sample size: 10538
[########################################] | 100% Completed | 101.70 ms
sample size: 10538
[########################################] | 100% Completed | 105.58 ms
[########################################] | 100% Completed | 101.79 ms
[########################################] | 100% Completed | 101.43 ms
[########################################] | 100% Completed | 104.55 ms
[########################################] | 100% Completed | 206.17 ms


In [34]:
m._constraint_engine


Constraint Engine 
 
Number of Constraints: 3
Number of Constraints Violated: 2 (66.66666666666667%)

GreaterThan 1 on columns [amount, balance]:  75 violated (7.5%) 
MeanBetweenRange on columns [amount, constant]: 0 (0.0%)
StandardDeviationBetween on columns [amount, constant]:  1 violated (50.0%) 

Remember that for **rows** constraints, the percentage refers to the number of violated rows (e.g. 7.5% for the constraint `GreaterThan`) while it refers to columns for **columns** constraints (e.g. 50% of columns are violated for the constraint `StandardDeviationBetween`).

# Custom constraints

The constraints engine comes with several pre-defined constraints to validate your dataset. 

However, in some cases, your constraints cannot be directly expressed by these default constraints. In thise case, you can create your own custom constraints from any Python function or lambda.

In [35]:
from ydata.constraints.rows import CustomConstraint

## A simple example

In the following example, we create a `CustomConstraint` to validate that each row is strictly lower than 10. Keep in mind that this is just a toy example and that in reality, the function could be arbitrarily complex depending on your own use-case:

In [44]:
c = CustomConstraint(name='Positive', columns=['constant', 'amount'], check=lambda x: x < 10)
mask = c.validate(dataset)
mask

,constant,amount
0,False,False
1,False,True
2,False,False
3,False,True
4,False,True
...,...,...
995,False,True
996,False,True
997,False,True
998,False,False


We can check the number of non-violated rows. As expected, all rows in `constaint` are violated while about 25% are violated for the columne `amount`.

In [45]:
mask.sum()

constant      0
amount      716
dtype: int64

We can easily select the violated rows for any columns. For instance, to select all violated rows for the column `amount`:

In [46]:
df[~mask['amount']]

,index,amount,constant,balance
0,0,15.134607,10,15.134607
2,2,20.515678,10,39.786009
5,5,13.179534,10,60.683651
6,6,11.621866,10,72.305517
7,7,26.016555,10,98.322073
...,...,...,...,...
980,980,12.388076,10,7311.747307
985,985,11.927358,10,7345.530319
989,989,23.876584,10,7386.815833
993,993,11.655929,10,7414.858869


## Custom constraints for series

A more complex example is the validation of time series when the rows have calculated values that depends on the previous rows.   
To handle such complex yet common use-cases, we provide a special synthax to refer to previous rows.

Imagine the following example: our dataset represents transactions on a bank account, and a structural constraint that **must** be respected is that the total amount at transaction **n** must be equal to the total amount at step **n-1** plus the amount of the transaction **n**.
If such constraint is violated it means that the data has some integrity issues. Although the example is fairly simple to understand, checking such integrity constraint with temporal dependency can be tricky.    

However, this is made simple with the `Constraint Engine` and the `Custom Constraint` as illustrated on the following example:

In [54]:
def check_balance_with_serie(df):
    return df['balance|n'] == df['balance|n-1'] + df['amount|n']  # 

In [55]:
# The custom synthax `column|n-k` is understood as the value `n-k` of the column `column` where `n` is the current row.
c = CustomConstraint(name="Balance Check with Serie", check=check_balance_with_serie,
                          available_columns=['balance|n', 'amount|n', 'balance|n-1'])

There is no restriction on the order of the temporal dependencies. For instance, `amount|n-5` would refer to the amount five rows behind the current one.

It is important to specify what colums you want to make available to your function using the parameter `available_columns` as above. It serves the purpose of preventing side effects and optimizing the validation of such constraint.

In [57]:
mask = c.validate(dataset)
mask.sum()

0    1000
dtype: int64

Because our dataset was constructed precisely to respect this constraint, we obtain no violated rows, as expected.

# Conclusion

The `Constraint Engine` is a versatile tool to validate constraints and detect issues in your dataset. Because structural constraints takes two forms, we offer two different types of constraints:

1. Rows constraints that check the validaty of each row,
2. Column constraints that check the validity of a column property.

When your constraint cannot be expressed with the default `Constraint` objects that we offer, it is possible to define `CustomConstraint` with no restriction, using Python function or lambda.

Finally, we also allow to define constraints with time dependency between the rows, allowing to check the integrity of complex relationships.